In [ ]:
# Setting parameters, replace with defaults when using a master notebook

countrycode = 'default'
countryname = 'default'

print("Parameters set")

# Import Libraries

In [ ]:
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.transform import from_origin, xy, Affine
from rasterio.features import geometry_mask
from rasterio.mask import mask
from rasterstats import zonal_stats
import numpy as np
import os
import time
from osgeo import gdal
gdal.UseExceptions()
from exactextract import exact_extract
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from shapely.geometry import shape, Point
from shapely.ops import unary_union
from shapely.geometry import box
import statsmodels.api as sm
import pyreadr
from geopy.distance import geodesic
import papermill as pm
import folium
import matplotlib.colors as mcolors
from PIL import Image, ImageDraw, ImageFont
import rioxarray
import argparse



print("All done.")

# Load and Prepare Data

In [ ]:
start_time = time.time()

# Setting the working directory to the country's subfolder
base_path = os.getcwd()
os.chdir(os.path.join(base_path, countrycode))

# Load the rds data
rds_file = countrycode + '_ntl_pop.rds'
results = pyreadr.read_r(rds_file)
# Extract the data frame from the result
data = results[None]  # Extracts the first (and usually only) dataframe from the result

print("Data loaded")

# Debugging: Check the first few rows of the data
print("Data preview before 'haslight' column creation:")
print(data.head(100))

# Create a binary 'haslight' column based on 'ntl' values
data['haslight'] = (data['ntl'] > 0).astype(int)

# Debugging: Check the first few rows after 'haslight' column creation
print("Data preview after 'haslight' column creation:")
print(data.head(100))

data.drop(columns=['ntl', 'pop'], inplace=True)
print("Columns 'ntl' and 'pop' dropped")


# Convert DataFrame to GeoDataFrame

Fast

In [ ]:
start_time = time.time()
# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data['x'], data['y']))
print("Converted to GeoDataFrame")
end_time = time.time()
print(f"Time to execute: {((time.time() - start_time)/60):.2f}")

In [ ]:
gdf.plot()

In [ ]:
ds = data.set_index(['y', 'x']).to_xarray()
ds.haslight.plot()

In [ ]:
# transform data array and write it to a GeoTIFF 

start_time = time.time()
data_array = ds.haslight.values[::-1]
xmin, ymin, xmax, ymax = gdf.total_bounds
pixel_size = 15 / 3600 
transform = Affine.translation(xmin - pixel_size / 2, ymax + pixel_size / 2) * Affine.scale(pixel_size, -pixel_size)#from_origin(xmin, ymax, pixel_size, pixel_size)
with rasterio.open(
    'haslight.tif',
    'w', 
    driver='GTiff',
    height=data_array.shape[0],
    width=data_array.shape[1],
    count=1,
    dtype=data_array.dtype,
    crs='EPSG:4326',
    transform=transform,
) as tiff:
    tiff.write(data_array, 1)
print(f"Time to execute: {((time.time() - start_time)/60):.2f}")

# Load Shapefile

This takes some time, depending on file size

In [ ]:
start_time = time.time()

# Go to the grid3 subfolder
grid3_folder = 'grid3_' + countrycode

# Find the shapefile in the directory
shapefile = None

#for countrynames with several words
first_word_countryname = countryname.split()[0]

# shapefile should contain the ending .shp, and the countrycode or countryname (at least the first word), as grid3 data tends to do
for file in os.listdir(grid3_folder):
    if file.endswith('.shp') and (countrycode in file or first_word_countryname in file):
        shapefile = file
        break

# Check if the shapefile was found and if it contains the countrycode
if shapefile is None:
    print('No shapefile found in the directory ending with .shp')

# Print the shapefile path
shapefile_path = os.path.join(grid3_folder, shapefile)

print("Loading")
print(shapefile_path)
print("...")

settlements = gpd.read_file(shapefile_path)

# Ensure spelling is Built-up and not Built-Up (e.g. Kenya data)
settlements['type'] = settlements['type'].replace('Built-Up Area', 'Built-up Area')

# Rename columns if necessary
columns_to_rename = {'pop_wp_un_adj': 'pop_un_adj', 'pop_wp_un_': 'pop_un_adj'}

for old_name, new_name in columns_to_rename.items():
    if old_name in settlements.columns and new_name not in settlements.columns:
        settlements.rename(columns={old_name: new_name}, inplace=True)
        print(f"Column '{old_name}' renamed to '{new_name}'")
        print(settlements.columns)
    else:
        if old_name not in settlements.columns:
            print(f"Column '{old_name}' does not exist.")
        if new_name in settlements.columns:
            print(f"Column '{new_name}' already exists.")


#checking total of settlements per type
settlement_type_counts = settlements['type'].value_counts()
print("Numbers for each settlement type before geometries applied:")
print(settlement_type_counts)

# Count the number of geometries for each settlement type before removal
type_counts_before = settlements['type'].value_counts()

# Ensure all geometries are valid
settlements['geometry'] = settlements['geometry'].astype(object).apply(lambda geom: geom if geom.is_valid else geom.buffer(0))

# Remove empty geometries
settlements = settlements[~settlements['geometry'].is_empty]

# Count the number of geometries for each settlement type after removal
type_counts_after = settlements['type'].value_counts()

# Calculate the number and proportion of geometries removed for each settlement type
for settlement_type in type_counts_before.index:
    count_before = type_counts_before[settlement_type]
    count_after = type_counts_after.get(settlement_type, 0)
    count_removed = count_before - count_after
    proportion_removed = count_removed / count_before
    print(f"{settlement_type}: {count_removed} geometries removed ({proportion_removed:.2%})")
print("Invalid and empty geometries removed")
print("Numbers for each settlement type AFTER geometries applied:")
settlement_type_counts = settlements['type'].value_counts()
print(settlement_type_counts)

print(f"Time to execute: {((time.time() - start_time)/60):.2f} minutes")


# Calculate zonal statistics

In this step, simply check if there is NTL (0/1), then for each settlement calculate what percentage of the polygon has NTL. 

This cell takes some time, with Sudan, around 10min.

In [ ]:
# Start time for execution time calculation
start_time = time.time()

# Specify the path to your raster data
raster_path = 'haslight.tif'

# Open the raster and convert it to binary (has light or has no light)
with rasterio.open(raster_path) as src:
    raster_data = src.read(1)  # Read the first band
    unique_values = np.unique(raster_data)
    print("Unique values in the original raster:", unique_values)
    
    threshold = 0  # Adjust this threshold as needed
    binary_raster_data = (raster_data > threshold).astype(int)  # Convert to binary

    # Write the binary raster to a new file (optional, can also be done in-memory)
    binary_raster_path = 'binary_haslight.tif'
    profile = src.profile
    with rasterio.open(binary_raster_path, 'w', **profile) as dst:
        dst.write(binary_raster_data, 1)

# Visualize the binary raster to ensure it looks correct
plt.imshow(binary_raster_data, cmap='gray')
plt.title("Binary Raster Data")
plt.show()

# Perform the exact extraction using the binary raster
settlements_updated = settlements.copy()
# Example of standardizing columns
required_columns = ['geometry', 'country', 'iso', 'type', 'pop_un_adj', 'adm0_pcode', 'adm1_name', 'adm2_name', 'settl_pcod']
settlements_updated = settlements_updated[required_columns]

# Check and align CRS
with rasterio.open(raster_path) as src:
    raster_crs = src.crs

if settlements_updated.crs != raster_crs:
    settlements_updated = settlements_updated.to_crs(raster_crs)
    print("CRS adjusted")

# Ensure geometries are valid and not empty
settlements_updated = settlements_updated[settlements_updated.is_valid]
settlements_updated = settlements_updated[~settlements_updated.is_empty]

# Check if geometries overlap with the raster bounds
with rasterio.open(binary_raster_path) as src:
    raster_bounds = src.bounds
    raster_polygon = box(*raster_bounds)

# Visualize settlement geometries on top of the binary raster
fig, ax = plt.subplots()
with rasterio.open(binary_raster_path) as src:
    binary_raster_data = src.read(1)
    plt.imshow(binary_raster_data, cmap='gray', extent=src.bounds)
settlements_updated.plot(ax=ax, facecolor='none', edgecolor='red')
plt.title("Settlement Geometries on Binary Raster")
plt.show()

# Perform exact extraction
results = exact_extract(binary_raster_path, settlements_updated, ["mean"])

# Add the weighted mean light coverage to the settlements GeoDataFrame
settlements_updated['NTL_weighted_percentage'] = [
    feature['properties'].get('mean', 0) * 100 if feature['properties'].get('mean') is not None else 0
    for feature in results
]
print("Weighted percentages added to GeoDataFrame")

# Check for NaN values in the GeoDataFrame
print("Number of NaN values in 'NTL_weighted_percentage':", settlements_updated['NTL_weighted_percentage'].isna().sum())

# Calculate the execution time
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time/60} minutes")


# Calculates distances


In [ ]:
# Start timing
start_time = time.time()

# Copy the GeoDataFrame with a new name
settlements_centroids = settlements_updated.copy()

# Dictionary mapping country codes to EPSG codes
country_epsg_mapping = {
    'AGO': 22033,
    'BDI': 32736, 'KEN': 32736, 'MOZ': 32736, 'RWA': 32736, 'TZA': 32736, 'UGA': 32736,
    'BFA': 32630,
    'CMR': 32633, 'CAF': 32633, 'TCD': 32633, 'COG': 32633, 'COD': 32633, 'NER': 32633, 'NGA': 32633,
    'ERI': 20136, 'ETH': 20136, 'SSD': 20136, 'SDN': 20136,
    'MLI': 32629,
    'SOM': 32639
}

# Assign EPSG code
country_epsg = country_epsg_mapping.get(countrycode, None)
if country_epsg:
    print(f"Country: {countrycode}, EPSG: {country_epsg}")
else:
    print(f"Country: {countrycode}, EPSG: Not Found")

# Reproject to a projected CRS (e.g., UTM zone 36N)
settlements_centroids = settlements_centroids.to_crs(epsg=country_epsg)

# Calculate the centroid of each geometry
settlements_centroids['centroid'] = settlements_centroids.geometry.centroid

# Reproject centroids back to geographic CRS (WGS84) for distance calculation
centroids_geo = settlements_centroids.copy()
centroids_geo = centroids_geo.set_geometry('centroid')
centroids_geo = centroids_geo.to_crs(epsg=4326)

# Filter out invalid centroids (latitude must be in the [-90; 90] range and longitude in the [-180; 180] range)
valid_centroids_geo = centroids_geo[
    (centroids_geo['centroid'].y >= -90) &
    (centroids_geo['centroid'].y <= 90) &
    (centroids_geo['centroid'].x >= -180) &
    (centroids_geo['centroid'].x <= 180)
]

invalid_count = len(centroids_geo) - len(valid_centroids_geo)

# Coordinates for Khartoum
khartoum_coords = (15.6, 32.5)

# Function to calculate the Haversine distance
def calculate_haversine_distance(centroid, destination_coords):
    centroid_coords = (centroid.y, centroid.x)  # (lat, lon)
    return geodesic(centroid_coords, destination_coords).kilometers

# Load the state capital coordinates from CSV
points_csv = 'state_capital_list_' + countrycode + '_points.csv'
state_capital_df = pd.read_csv(points_csv)

# Convert the state capital coordinates to a dictionary
state_capital_dict = state_capital_df.set_index('state')['state_capital_point'].to_dict()

# Function to calculate the distance to the state capital
def calculate_state_capital_distance(row):
    state = row['adm1_name']
    if state in state_capital_dict:
        state_capital_coords = tuple(map(float, state_capital_dict[state].strip('()').split(',')))
        return calculate_haversine_distance(row['centroid'], state_capital_coords)
    else:
        return None

# Calculate the distance to the state capital
valid_centroids_geo['state_capital_distance'] = valid_centroids_geo.apply(
    calculate_state_capital_distance, axis=1
)

# Print the count of invalid entries
print(f"Number of invalid entries excluded: {invalid_count}")

# Print the first few rows to verify
print(valid_centroids_geo[['centroid', 'state_capital_distance', 'NTL_weighted_percentage']].head())

print(f"Time to execute: {((time.time() - start_time)/60):.2f} minutes")


# Save Results


In [ ]:
start_time = time.time()

csv_name = countrycode + '_grid3_with_ntl_percentage.csv'
valid_centroids_geo.drop(columns=['geometry']).to_csv(csv_name, index=False)
print("CSV saved, all done.")

print(f"Time to execute: {((time.time() - start_time)/60):.2f} minutes")


# Calculate key stats

In [ ]:
# Load the data from the CSV file
df = pd.read_csv(csv_name)

print(f"\033[1mKey stats {countryname}n\033[0m")

# Calculate 'something_wrong' (should be 0)
#something_wrong = (df['NTL_weighted_percentage'] > 100).sum()
#print(f"Number of rows where there is something wrong with 'NTL_weighted_percentage': {something_wrong}")
something_wrong = (df['NTL_weighted_percentage'] > 100).sum()
print(f"Something wrong with {something_wrong} entries")
      
print(" ")
# Calculate 'settl_ntl_perc'
print("\033[1mOverview all settlements\033[0m")
settl_ntl_perc = (df['NTL_weighted_percentage'] > 0).mean()
print(f"Proportion nighttime light: {settl_ntl_perc:.2%}")
settl_monit_perc = (df['NTL_weighted_percentage'] > 50).mean()
print(f"Proportion monitorable: {settl_monit_perc:.2%}")
anyntl = df[df['NTL_weighted_percentage'] > 0]
all_median_any = anyntl['NTL_weighted_percentage'].median()

print(" ")
print("\033[1mOverview built-up areas\033[0m")
# Calculate 'built_up_perc'
df_built_up = df[df['type'] == 'Built-up Area']
#Get the total number of built-up areas; shape gives tuple of dimensions
print(f"Total number of built-up areas: {df_built_up.shape[0]}")
built_ntl_perc = (df_built_up['NTL_weighted_percentage'] > 0).mean()
print(f"Proportion nighttime light > 0: {built_ntl_perc:.2%}")

#median ntl coverage of all the settlements that have at least some NTL
anyntl = df_built_up[df_built_up['NTL_weighted_percentage'] > 0]
built_median_any = anyntl['NTL_weighted_percentage'].median()
print(f"Median NTL for settlements with any NTL: {built_median_any}")
built_monit_perc = (df_built_up['NTL_weighted_percentage'] > 50).mean()
print(f"Proportion monitorable: {built_monit_perc:.2%}")


print(" ")
print("\033[1mOverview small settlements\033[0m")
# Calculate 'small_settl_perc'
df_small_set = df[df['type'] == 'Small Settlement Area']
print(f"Total number of small settlement areas: {df_small_set.shape[0]}")
smallset_ntl_perc = (df_small_set['NTL_weighted_percentage'] > 0).mean()
print(f"Proportion nighttime light: {smallset_ntl_perc:.2%}")
#median ntl coverage of all the settlements that have at least some NTL
anyntl = df_small_set[df_small_set['NTL_weighted_percentage'] > 0]
small_median_any = anyntl['NTL_weighted_percentage'].median()
print(f"Median NTL for settlements with any NTL: {small_median_any}")
smallset_monit_perc = (df_small_set['NTL_weighted_percentage'] > 50).mean()
print(f"Proportion monitorable: {smallset_monit_perc:.2%}")

print(" ")
print("\033[1mOverview hamlets\033[0m")
# Calculate 'hamlets_perc'
df_hamlet = df[df['type'] == 'Hamlet']
print(f"Total number of hamlets: {df_hamlet.shape[0]}")
haml_ntl_perc = (df_hamlet['NTL_weighted_percentage'] > 0).mean()
print(f"Proportion nighttime light: {haml_ntl_perc:.2%}")
#median ntl coverage of all the settlements that have at least some NTL
anyntl = df_hamlet[df_hamlet['NTL_weighted_percentage'] > 0]
hamlet_median_any = anyntl['NTL_weighted_percentage'].median()
print(f"Median NTL for settlements with any NTL: {hamlet_median_any}")
haml_monit_perc = (df_hamlet['NTL_weighted_percentage'] > 50).mean()
print(f"Proportion monitorable: {haml_monit_perc:.2%}")


#settlements with more than 50% ntl
df_mont = df[df['NTL_weighted_percentage'] > 50]

#settlements with any ntl
df_anyntl = df[df['NTL_weighted_percentage'] > 0]


# save stats to CSV

In [ ]:
# Create a list of dictionaries
stats = [
    {"type": "Error", "what": "Settlements with over 100% NTL", "value": something_wrong},
    {"type": "All settlements", "what": "Total number", "value": len(df)},
    {"type": "All settlements", "what": "Proportion NTL", "value": f"{settl_ntl_perc:.2%}"},
    {"type": "All settlements", "what": "Proportion monitorable", "value": f"{settl_monit_perc:.2%}"},
    {"type": "All settlements", "what": "Median size NTL settlement", "value": f"{all_median_any}"},


    {"type": "Built-up", "what": "Total number", "value": df_built_up.shape[0]},
    {"type": "Built-up", "what": "Proportion NTL", "value": f"{built_ntl_perc:.2%}"},
    {"type": "Built-up", "what": "Median NTL value for any NTL", "value": f"{built_median_any}"},
    {"type": "Built-up", "what": "Proportion monitorable", "value": f"{built_monit_perc:.2%}"},

    {"type": "Small settlements", "what": "Total number", "value": df_small_set.shape[0]},
    {"type": "Small settlements", "what": "Proportion NTL", "value": f"{smallset_ntl_perc:.2%}"},
    {"type": "Small settlements", "what": "Median NTL value for any NTL", "value": f"{small_median_any}"},
    {"type": "Small settlements", "what": "Proportion monitorable", "value": f"{smallset_monit_perc:.2%}"},

    {"type": "Hamlets", "what": "Total number", "value": df_hamlet.shape[0]},
    {"type": "Hamlets", "what": "Proportion NTL", "value": f"{haml_ntl_perc:.2%}"},
    {"type": "Hamlets", "what": "Median NTL value for any NTL", "value": f"{hamlet_median_any}"},
    {"type": "Hamlets", "what": "Proportion monitorable", "value": f"{haml_monit_perc:.2%}"},
]

# Convert the list of dictionaries into a DataFrame
df_output = pd.DataFrame(stats)

# Save the DataFrame to a CSV file
output_file = 'overview_key_stats.csv'
df_output.to_csv(output_file, index=False)

print("Data saved")


# delete files that are no longer needed

In [ ]:
# Define the file paths
files_to_delete = ['binary_haslight.tif', 'haslight.tif']

# Loop through the files and delete them
for file in files_to_delete:
    if os.path.exists(file):
        os.remove(file)
        print(f"Deleted file: {file}")
    else:
        print(f"File not found: {file}")
print("All done.")